In [1]:
import pandas as pd
import re


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\kim\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\kim\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\kim\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.sta

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\kim\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\kim\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\kim\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.sta

AttributeError: _ARRAY_API not found

In [2]:
linespeed = pd.read_excel('25년 생산 제품 공정정보(1월~7월)-베합코드 추가.xlsx', sheet_name = '25년 공정정보(1월~7월) ', skiprows = 1)

In [3]:
operation_sequence = linespeed[['GITEMNO', '공정명', '공정명.1', '공정명.2', '공정명.3', '공정명.4', '공정명.5', '공정명.6']]
new_cols = ['GITEM', '1공정', '2공정', '3공정', '4공정', '5공정', '6공정', '7공정']  # 컬럼 수와 길이 일치시켜야 함
operation_sequence.columns = new_cols

operation_sequence

,GITEM,1공정,2공정,3공정,4공정,5공정,6공정,7공정
0,31940,Pattern 성형,투명점착,NaN,NaN,NaN,NaN,NaN
1,32452,TPU PATTERN성형,NaN,NaN,NaN,NaN,NaN,NaN
2,32540,안료접착,MIBK SR,투명점착,인쇄,NaN,NaN,NaN
3,32405,TOP COATING,투명접착,투명점착,NaN,NaN,NaN,NaN
4,32746,PEARL EPU코팅,BLACK EPU코팅,PPF점착,TOP COATING,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
287,32470,PPF점착,TOP COATING,NaN,NaN,NaN,NaN,NaN
288,32041,투명점착,TOP COATING,NaN,NaN,NaN,NaN,NaN
289,32534,PPF점착,TOP COATING,NaN,NaN,NaN,NaN,NaN
290,32571,EPU코팅,WHITE EPU코팅,투명점착,TOP COATING,NaN,NaN,NaN


In [4]:
id_vars = ['GITEMNO', 'GitemName']

all_cols = linespeed.columns
variable_prefixes = set([re.sub(r'\.\d+$', '', col) for col in all_cols if col not in id_vars]) # id_vars에 존재하지 않는 컬럼 중 .(숫자) 존재하지 않는 컬럼)

long_linespeed = []

for i in range(7): 
    temp_cols = []
    rename_mapping = {}
    
    for prefix in variable_prefixes:
        col_name = f"{prefix}" if i == 0 else f"{prefix}.{i}"
        if col_name in linespeed.columns:
            temp_cols.append(col_name)
            rename_mapping[col_name] = prefix 

    temp_linespeed = linespeed[id_vars + temp_cols].copy()
    temp_linespeed = temp_linespeed.rename(columns=rename_mapping)
    temp_linespeed['공정순서'] = i+1 
    
    long_linespeed.append(temp_linespeed)

melted_linespeed = pd.concat(long_linespeed, ignore_index=True)

melted_linespeed = melted_linespeed.sort_values(['GITEMNO', '공정순서']).reset_index(drop=True)

# 공정 순서 없는 경우 삭제
melted_linespeed = melted_linespeed.dropna(subset = {'수율', '공정코드', '합지Doff(Gitem)', '합지BF2', '합지BF', '호기코드', '투입BF', '공정명'}, how = 'all')
melted_linespeed

,GITEMNO,GitemName,소분류,수율,중분류,합지Doff(Gitem),배합코드 이력,합지BF,L/Speed,공정명,합지BF2,공정코드,투입BF,호기코드,배합코드,공정순서
0,25026,SCF-NS-SS03,1PLY,98.54,염료,NaN,W2017,23352,29.8,염료점착,NaN,20300.0,23200,"C2010, C2260",W2017,1
1,25026,SCF-NS-SS03,NaN,99.16,NaN,NaN,"W1809, T0873, T02149",NaN,47.8,유광 S/R,NaN,20700.0,23200,"C2270, C2010",T02149,2
7,25029,SCF-NS-SS19,1PLY,98.89,염료,NaN,W2018,23352,30.0,염료점착,NaN,20300.0,23200,C2260,W2018,1
8,25029,SCF-NS-SS19,NaN,99.42,NaN,NaN,"T02149, W1809",NaN,47.0,유광 S/R,NaN,20700.0,23200,"C2270, C2010",W1809,2
14,25030,SCF-NS-SS35,1PLY,99.02,염료,NaN,W2019,23352,30.0,염료점착,NaN,20300.0,23200,C2260,W2019,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2033,32745,PPF-NS-Onyxhield Exa desert sand,NaN,96.39,NaN,31729.0,NaN,NaN,10.0,TOP COATING,NaN,12400.0,25178,C2010,T01702,4
2037,32746,PPF-NS-Onyxhield Exa Viola-30,C-PPF,89.78,HIGH-END,NaN,T02533,26386,10.0,PEARL EPU코팅,NaN,23315.0,25628,C2010,T02533,1
2038,32746,PPF-NS-Onyxhield Exa Viola-30,NaN,95.12,NaN,NaN,T01621,26386,10.0,BLACK EPU코팅,NaN,20920.0,25628,C2010,T01621,2
2039,32746,PPF-NS-Onyxhield Exa Viola-30,NaN,50.43,NaN,NaN,T01514,25800,10.0,PPF점착,NaN,23311.0,25628,C2010,T01514,3


In [5]:
yield_data = melted_linespeed[['GITEMNO', '공정명', '호기코드', '수율']]
yield_data = yield_data.rename(columns = {"GITEMNO":"GITEM"})
yield_data['수율'] = yield_data['수율'] /100

In [6]:
melted_linespeed = melted_linespeed[['GITEMNO', 'L/Speed','호기코드', '공정명', '공정코드', '배합코드', '공정순서']]
melted_linespeed['호기코드'] = melted_linespeed['호기코드'].str.split(', ')
melted_linespeed = melted_linespeed.explode('호기코드').reset_index(drop=True)
melted_linespeed

C:\Users\kim\AppData\Local\Temp\ipykernel_45884\608877047.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  melted_linespeed['호기코드'] = melted_linespeed['호기코드'].str.split(', ')


,GITEMNO,L/Speed,호기코드,공정명,공정코드,배합코드,공정순서
0,25026,29.8,C2010,염료점착,20300.0,W2017,1
1,25026,29.8,C2260,염료점착,20300.0,W2017,1
2,25026,47.8,C2270,유광 S/R,20700.0,T02149,2
3,25026,47.8,C2010,유광 S/R,20700.0,T02149,2
4,25029,30.0,C2260,염료점착,20300.0,W2018,1
...,...,...,...,...,...,...,...
1117,32745,10.0,C2010,TOP COATING,12400.0,T01702,4
1118,32746,10.0,C2010,PEARL EPU코팅,23315.0,T02533,1
1119,32746,10.0,C2010,BLACK EPU코팅,20920.0,T01621,2
1120,32746,10.0,C2010,PPF점착,23311.0,T01514,3


In [7]:
gitem_sequence = melted_linespeed[['GITEMNO',  '공정코드', '공정명', '배합코드', '공정순서']].drop_duplicates()
gitem_sequence 

,GITEMNO,공정코드,공정명,배합코드,공정순서
0,25026,20300.0,염료점착,W2017,1
2,25026,20700.0,유광 S/R,T02149,2
4,25029,20300.0,염료점착,W2018,1
5,25029,20700.0,유광 S/R,W1809,2
7,25030,20300.0,염료점착,W2019,1
...,...,...,...,...,...
1117,32745,12400.0,TOP COATING,T01702,4
1118,32746,23315.0,PEARL EPU코팅,T02533,1
1119,32746,20920.0,BLACK EPU코팅,T01621,2
1120,32746,23311.0,PPF점착,T01514,3


In [8]:
# melted_linespeed['배합코드'].fillna("배합코드없음", inplace = True)

machine_info_pivot = melted_linespeed.pivot_table(
    index = ['GITEMNO','공정명'], #  '공정코드', '배합코드', '공정순서'제외 
    columns = "호기코드", 
    values = "L/Speed",
).reset_index().rename_axis(None, axis=1)

# machine_info_pivot['배합코드'].replace('배합코드없음', None, inplace=True)

machine_info_pivot

,GITEMNO,공정명,C2010,C2250,C2260,C2270,O2310,O2340,O2360,O2510
0,25026,염료점착,29.8,NaN,29.8,NaN,NaN,NaN,NaN,NaN
1,25026,유광 S/R,47.8,NaN,NaN,47.8,NaN,NaN,NaN,NaN
2,25029,염료점착,NaN,NaN,30.0,NaN,NaN,NaN,NaN,NaN
3,25029,유광 S/R,47.0,NaN,NaN,47.0,NaN,NaN,NaN,NaN
4,25030,염료점착,NaN,NaN,30.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
992,32745,안료점착,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
993,32746,BLACK EPU코팅,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
994,32746,PEARL EPU코팅,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
995,32746,PPF점착,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# 컬럼명을 기존의 코드에 맞춰서 변경.
for df in [machine_info_pivot, gitem_sequence]:
    df.rename(columns = {"GITEMNO":"GITEM",
                        "공정명":"공정",
                        "공정코드":"공정code",
                        "소분류":"제품분류"}, inplace=True)

In [10]:
import pandas as pd

machine_master_info = [
    {"기계코드": "C2010", "기계인덱스": 0, "기계이름": "1호기"},
    {"기계코드": "C2250", "기계인덱스": 1, "기계이름": "25호기"},
    {"기계코드": "C2260", "기계인덱스": 2, "기계이름": "26호기"},
    {"기계코드": "C2270", "기계인덱스": 3, "기계이름": "27호기"},
    {"기계코드": "O2310", "기계인덱스": 4, "기계이름": "31호기"},
    {"기계코드": "O2340", "기계인덱스": 5, "기계이름": "34호기"},
    {"기계코드": "O2360", "기계인덱스": 6, "기계이름": "36호기"},
    {"기계코드": "O2510", "기계인덱스": 7, "기계이름": "51호기"},
]

machine_master_df = pd.DataFrame(machine_master_info, columns=["기계코드", "기계인덱스", "기계이름"])


In [11]:
machine_info_pivot = machine_info_pivot.rename(columns = {"공정": "공정명"})
gitem_sequence = gitem_sequence.rename(columns = {"공정": "공정명", "공정code" : "공정코드"})
yield_data = yield_data.rename(columns = {"호기코드": "기계코드"})

In [12]:
with pd.ExcelWriter('품목별 분리 라인스피드 및 공정 순서.xlsx') as writer:
    machine_info_pivot.to_excel(writer, sheet_name="품목별 라인스피드", index=False)
    gitem_sequence.to_excel(writer, sheet_name="공정순서", index=False)
    yield_data.to_excel(writer, sheet_name="수율데이터", index=False)
    operation_sequence.to_excel(writer, sheet_name="GITEM별 공정", index=False)
    machine_master_df.to_excel(writer, sheet_name="기계기준정보", index=False)

In [13]:
order = pd.read_excel("25년 5월 PO 내역(송부건).xlsx", sheet_name="25년 5월 PO확정 내역")
print(f"원본 주문 데이터 로딩: {len(order)}건")

# SPEC 컬럼 분리 (두께*너비*길이)
order[["두께", "너비", "길이"]] = order["SPEC"].str.split("*", expand=True).astype(float)

# 표준 너비 조정 (914.4 → 914.0)
adjusted_count = (order["너비"] == 914.4).sum()
order.loc[order["너비"] == 914.4, "너비"] = 914.0
print(f"표준 너비 조정: {adjusted_count}건 (914.4 → 914.0)")

# 원단길이 계산 (길이 * 의뢰량)
order["원단길이"] = order["길이"] * order["의뢰량"]

# 필요한 컬럼만 선택
processed_order = order[["P/O NO", "GITEM", "GITEM명", "너비", "길이", "의뢰량", "원단길이", "납기일"]]

# 전처리된 데이터 저장
processed_order.to_excel("preprocessed_order.xlsx", index=False)
print(f"✅ 전처리 완료! {len(processed_order)}건 → preprocessed_order.xlsx")


FileNotFoundError: [Errno 2] No such file or directory: '25년 5월 PO 내역(송부건).xlsx'